<a href="https://colab.research.google.com/github/fambargh/projectmastery/blob/master/unet_keras_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
pip install keras-segmentation


In [3]:
from zipfile import ZipFile
filename = "image.zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [4]:
from zipfile import ZipFile
filename = "annotate_cup3.zip"

with ZipFile(filename,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [70]:
!python -m keras_segmentation verify_dataset \
 --images_path="/content/image_train/" \
 --segs_path="/content/annotate_cup3/annotate_cup_train/"  \
 --n_classes=2

Using TensorFlow backend.
  0% 0/178 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/keras_segmentation/__main__.py", line 4, in <module>
    cli_interface.main()
  File "/usr/local/lib/python3.6/dist-packages/keras_segmentation/cli_interface.py", line 136, in main
    cli_verify_dataset()
  File "/usr/local/lib/python3.6/dist-packages/keras_segmentation/cli_interface.py", line 105, in cli_verify_dataset
    data_utils.data_loader.verify_segmentation_dataset( args.images_path , args.segs_path , args.n_classes )
  File "/usr/local/lib/python3.6/dist-packages/keras_segmentation/data_utils/data_loader.py", line 106, in verify_segmentation_dataset
    assert ( np.max(seg[:,:,0]) < n_classes) , "The pixel values of seg image should be from 0 to "+str(n_classe

In [71]:
!python -m keras_segmentation visualize_dataset \
 --images_path="dataset_path/images_prepped_train/" \
 --segs_path="dataset_path/annotations_prepped_train/"  \
 --n_classes=2

Using TensorFlow backend.
Press any key to navigate. 
visualize_dataset


In [0]:
import keras_segmentation

model = keras_segmentation.models.unet.vgg_unet(n_classes=256 ,  input_height=128, input_width=128  )

In [73]:
model.train( 
    train_images =  "/content/image_train/",
    train_annotations = "/content/annotate_cup3/annotate_cup_train/",
    checkpoints_path = "/content/checkpoint/vgg_unet_1" , epochs=5
)

100%|██████████| 178/178 [00:00<00:00, 1142.34it/s]

Verifying train dataset
Dataset verified! 
Starting Epoch  0


Epoch 1/1
512/512 [==============================] - 58s 113ms/step - loss: 0.5731 - acc: 0.9138
saved  /content/checkpoint/vgg_unet_1.model.0
Finished Epoch 0
Starting Epoch  1
Epoch 1/1
512/512 [==============================] - 55s 108ms/step - loss: 0.1319 - acc: 0.9502
saved  /content/checkpoint/vgg_unet_1.model.1
Finished Epoch 1
Starting Epoch  2
Epoch 1/1
512/512 [==============================] - 55s 108ms/step - loss: 0.1117 - acc: 0.9557
saved  /content/checkpoint/vgg_unet_1.model.2
Finished Epoch 2
Starting Epoch  3
Epoch 1/1
512/512 [==============================] - 56s 109ms/step - loss: 0.1000 - acc: 0.9595
saved  /content/checkpoint/vgg_unet_1.model.3
Finished Epoch 3
Starting Epoch  4
Epoch 1/1
512/512 [==============================] - 56s 109ms/step - loss: 0.0903 - acc: 0.9631
saved  /content/checkpoint/vgg_unet_1.model.4
Finished Epoch 4


In [0]:
pip install keras-segmentation


In [0]:
from keras import Input
from keras import layers
#from keras import Dropout
from keras.layers import UpSampling2D

In [20]:
img_input = Input(shape=(128,128 , 3 ))

conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(img_input)
conv1 = layers.Dropout(0.2)(conv1)
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = layers.Dropout(0.2)(conv2)
conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = layers.Dropout(0.2)(conv3)
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

up1 = layers.concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = layers.Dropout(0.2)(conv4)
conv4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = layers.concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = layers.Dropout(0.2)(conv5)
conv5 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
out = layers.Conv2D( 256, (1, 1) , padding='same')(conv5)

from keras_segmentation.models.model_utils import get_segmentation_model

model = get_segmentation_model(img_input ,  out ) # this would build the segmentation model

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [21]:

model.train( 
    train_images =  ,
    train_annotations = "/content/annotate_cup3/annotate_cup_train/",
    checkpoints_path = "/content/checkpoint/vgg_unet_1" , epochs=2
)

100%|██████████| 178/178 [00:00<00:00, 1233.93it/s]

Verifying train dataset
Dataset verified! 
Starting Epoch  0


Epoch 1/1
512/512 [==============================] - 72s 141ms/step - loss: 1.7110 - acc: 0.8910
saved  /content/checkpoint/vgg_unet_1.model.0
Finished Epoch 0
Starting Epoch  1
Epoch 1/1
512/512 [==============================] - 70s 138ms/step - loss: 1.7145 - acc: 0.8936
saved  /content/checkpoint/vgg_unet_1.model.1
Finished Epoch 1


In [0]:
import os
f='/content/image_test/'
o=os.listdir(f)

In [82]:
o

['drishtiGS_099.png',
 'S-12-L.png',
 'S-20-L.png',
 'S-31-L.png',
 'drishtiGS_101.png',
 'S-25-L.png',
 'S-14-R.png',
 'S-28-R.png',
 'S-15-L.png',
 'S-32-L.png',
 'drishtiGS_095.png',
 'drishtiGS_092.png',
 'S-33-R.png',
 'drishtiGS_098.png',
 'drishtiGS_088.png',
 'S-21-R.png',
 'S-30-L.png',
 'drishtiGS_089.png',
 'drishtiGS_087.png',
 'S-18-L.png',
 'drishtiGS_091.png',
 'S-26-R.png',
 'S-27-L.png',
 'drishtiGS_093.png',
 'drishtiGS_090.png',
 'S-24-R.png',
 'S-22-L.png',
 'S-23-L.png',
 'S-13-R.png',
 'S-29-R.png',
 'S-16-R.png',
 'S-19-R.png',
 'S-34-L.png',
 'drishtiGS_096.png',
 'drishtiGS_097.png',
 'drishtiGS_100.png',
 'S-17-L.png',
 'S-35-R.png']

In [0]:
images_dir ='/content/image_test'
ids = os.listdir(images_dir)
images_fps = [os.path.join(images_dir, image_id) for image_id in ids]

In [84]:
images_fps

['/content/image_test/drishtiGS_099.png',
 '/content/image_test/S-12-L.png',
 '/content/image_test/S-20-L.png',
 '/content/image_test/S-31-L.png',
 '/content/image_test/drishtiGS_101.png',
 '/content/image_test/S-25-L.png',
 '/content/image_test/S-14-R.png',
 '/content/image_test/S-28-R.png',
 '/content/image_test/S-15-L.png',
 '/content/image_test/S-32-L.png',
 '/content/image_test/drishtiGS_095.png',
 '/content/image_test/drishtiGS_092.png',
 '/content/image_test/S-33-R.png',
 '/content/image_test/drishtiGS_098.png',
 '/content/image_test/drishtiGS_088.png',
 '/content/image_test/S-21-R.png',
 '/content/image_test/S-30-L.png',
 '/content/image_test/drishtiGS_089.png',
 '/content/image_test/drishtiGS_087.png',
 '/content/image_test/S-18-L.png',
 '/content/image_test/drishtiGS_091.png',
 '/content/image_test/S-26-R.png',
 '/content/image_test/S-27-L.png',
 '/content/image_test/drishtiGS_093.png',
 '/content/image_test/drishtiGS_090.png',
 '/content/image_test/S-24-R.png',
 '/content/im

In [0]:
out = model.predict_segmentation(
    inp=images_fps[1],
    out_fname="/content/output/{}.png"
)

In [0]:
for i in range(len(images_fps)):
  out = model.predict_segmentation(
      inp=images_fps[i],
      out_fname='/content/output/o[{}].png'.format(i)
      #out_fname="/content/output/{}.png".format(i)
      )


In [43]:
images_fps[1]

'/content/image_test/S-12-L.png'

In [0]:
!zip -r /content/output.zip /content/output